In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, SpatialDropout1D, SimpleRNN
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D, RNN
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint

2022-07-21 16:59:58.734056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jiwon/study/catkin_ws/devel/lib:/home/jiwon/study/ws_moveit/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2022-07-21 16:59:58.734082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def calc_change_pct(open, close):
    return (round((close - open) / open, 4) * 100)

In [3]:
# Drop unnecessary columns
stock = pd.read_csv('../csv/TSLA_addStockComp.csv')
stock = stock.rename(columns={'gap':'Gap', 'upDown':'Signal'})

stock_binary = stock.drop(['Open', 'Close'], axis=1)
stock_binary.to_csv('../csv/binary_target.csv', index=False)

In [4]:
stock_binary.head()

,Date,Volume,Gap,Signal
0,2010-06-29,93831500,25.73,1
1,2010-06-30,85935500,-7.59,-1
2,2010-07-01,41094000,-12.15,-1
3,2010-07-02,25699000,-16.52,-1
4,2010-07-06,34334500,-19.45,-1


In [5]:
text = pd.read_csv('../csv/combined_text_data.csv', dtype={'Retweets':'str'})
text = text.drop(['ID'], axis=1)

In [6]:
# Merge tweet dataset + stock dataset
take_2 = pd.merge(text, stock_binary, on='Date', how='left')
take_2 = take_2.set_index('Date')
take_2.index = pd.DatetimeIndex(take_2.index)
take_2.head()

,Tweet,Retweets,Tweet Length,Sentiment,Polarity,abandon,abdomen,abigail,ability,ablate,...,yr,zero,zip,zombie,zone,zoo,zoom,Volume,Gap,Signal
Date,,,,,,,,,,,,,,,,,,,,,
2010-06-05,Please ignore prior tweets as that was someone...,"5,453",106,0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2011-12-01,Went to Iceland on Sat to ride bumper cars on ...,188,129,1,0.65,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5151000.0,0.09,0.0
2011-12-01,I made the volume on the Model S,78,125,0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5151000.0,0.09,0.0
2011-12-03,Great Voltaire quote arguably better than Twai...,44,145,1,0.45,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2011-12-03,That was a total non sequitur btw,50,42,0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from gensim.models import Word2Vec

In [8]:
le = LabelEncoder()
pca = PCA(n_components=6551)

# Fit PCA
tfidf_df = take_2.iloc[:, 5:-3]
pca.fit(tfidf_df)

PCA(n_components=6551)

In [9]:
# Transform PCA into DataFrame
x_pca = pca.transform(tfidf_df)

pca_df = pd.DataFrame(x_pca)
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,...,6541,6542,6543,6544,6545,6546,6547,6548,6549,6550
0,-0.105592,-0.029830,-0.083962,0.082105,0.017019,0.134082,-0.072522,-0.006849,-0.068842,0.106781,...,2.670864e-17,-1.728328e-17,-6.740265e-18,3.699205e-17,1.521779e-17,-3.403527e-17,-1.770595e-17,-1.371855e-17,3.456318e-17,-2.172258e-17
1,-0.099076,-0.021082,-0.038139,0.000626,0.003329,-0.062547,0.042404,0.019719,0.089294,-0.006209,...,-6.527914e-17,1.850555e-17,-1.612357e-16,-1.365654e-16,-1.207107e-17,-4.292488e-17,-1.012632e-16,2.789449e-17,2.114618e-18,-6.527066e-17
2,-0.114546,-0.035896,-0.126653,0.046923,0.045059,-0.215881,0.223829,-0.022574,-0.048085,-0.125213,...,-1.050660e-17,-7.721976e-18,2.180771e-17,-2.343465e-17,8.491505e-18,-1.573575e-18,-5.592959e-18,-1.693134e-17,-1.008393e-17,2.044145e-17
3,-0.091030,-0.017803,0.093363,0.155490,0.025837,-0.022723,0.038680,-0.032345,-0.094818,0.016137,...,-4.191797e-17,1.291852e-17,-2.013440e-17,7.199780e-21,8.755356e-17,5.221323e-18,-8.323073e-17,5.480303e-18,-7.318216e-17,-1.918148e-17
4,-0.083948,-0.008909,0.008896,0.003783,0.006889,0.062488,-0.047705,-0.032693,-0.075397,0.037750,...,2.290377e-17,-2.852849e-17,3.189079e-18,-2.097762e-17,1.319635e-16,1.892208e-17,-5.606935e-18,1.346528e-17,8.203938e-18,1.498825e-18


In [10]:
# Labeling words as pca_i instead of word itself
labels = []
for i in range(1, 6552):
    labels.append('pc_'+ str(i))

pca_df.columns = labels

In [11]:
# Replace empty word columns with processed PCA
take_2 = take_2.reset_index()
df_with_tfidf_pca = pd.concat([take_2.iloc[:, :6], pca_df], axis=1)
df_with_tfidf_pca.drop(columns = ['Sentiment'], inplace=True)

df_with_tfidf_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12191 entries, 0 to 12190
Columns: 6556 entries, Date to pc_6551
dtypes: datetime64[ns](1), float64(6552), int64(1), object(2)
memory usage: 609.8+ MB


In [12]:
df_with_tfidf_pca = pd.concat([df_with_tfidf_pca, take_2.iloc[:, -3:]], axis=1)
df_with_tfidf_pca.head()

,Date,Tweet,Retweets,Tweet Length,Polarity,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_6545,pc_6546,pc_6547,pc_6548,pc_6549,pc_6550,pc_6551,Volume,Gap,Signal
0,2010-06-05,Please ignore prior tweets as that was someone...,"5,453",106,0.00,-0.105592,-0.029830,-0.083962,0.082105,0.017019,...,3.699205e-17,1.521779e-17,-3.403527e-17,-1.770595e-17,-1.371855e-17,3.456318e-17,-2.172258e-17,NaN,NaN,NaN
1,2011-12-01,Went to Iceland on Sat to ride bumper cars on ...,188,129,0.65,-0.099076,-0.021082,-0.038139,0.000626,0.003329,...,-1.365654e-16,-1.207107e-17,-4.292488e-17,-1.012632e-16,2.789449e-17,2.114618e-18,-6.527066e-17,5151000.0,0.09,0.0
2,2011-12-01,I made the volume on the Model S,78,125,0.00,-0.114546,-0.035896,-0.126653,0.046923,0.045059,...,-2.343465e-17,8.491505e-18,-1.573575e-18,-5.592959e-18,-1.693134e-17,-1.008393e-17,2.044145e-17,5151000.0,0.09,0.0
3,2011-12-03,Great Voltaire quote arguably better than Twai...,44,145,0.45,-0.091030,-0.017803,0.093363,0.155490,0.025837,...,7.199780e-21,8.755356e-17,5.221323e-18,-8.323073e-17,5.480303e-18,-7.318216e-17,-1.918148e-17,NaN,NaN,NaN
4,2011-12-03,That was a total non sequitur btw,50,42,0.00,-0.083948,-0.008909,0.008896,0.003783,0.006889,...,-2.097762e-17,1.319635e-16,1.892208e-17,-5.606935e-18,1.346528e-17,8.203938e-18,1.498825e-18,NaN,NaN,NaN


In [13]:
df_with_tfidf_pca = df_with_tfidf_pca.dropna()
df_with_tfidf_pca.head()

,Date,Tweet,Retweets,Tweet Length,Polarity,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_6545,pc_6546,pc_6547,pc_6548,pc_6549,pc_6550,pc_6551,Volume,Gap,Signal
1,2011-12-01,Went to Iceland on Sat to ride bumper cars on ...,188,129,0.650000,-0.099076,-0.021082,-0.038139,0.000626,0.003329,...,-1.365654e-16,-1.207107e-17,-4.292488e-17,-1.012632e-16,2.789449e-17,2.114618e-18,-6.527066e-17,5151000.0,0.09,0.0
2,2011-12-01,I made the volume on the Model S,78,125,0.000000,-0.114546,-0.035896,-0.126653,0.046923,0.045059,...,-2.343465e-17,8.491505e-18,-1.573575e-18,-5.592959e-18,-1.693134e-17,-1.008393e-17,2.044145e-17,5151000.0,0.09,0.0
6,2011-12-21,Got called randomly by Kanye West today and re...,"1,682",149,-0.500000,-0.025252,-0.022537,-0.027629,0.010788,0.008821,...,3.543562e-18,1.498232e-16,-3.937284e-17,-7.610591e-17,1.666605e-16,-1.574024e-16,-1.107441e-16,8527500.0,-1.21,-1.0
7,2011-12-22,Model S options are out Performance in red and...,697,108,-0.083333,-0.010046,-0.037306,-0.072366,0.018803,-0.022556,...,7.314553e-18,-4.216318e-17,-4.045916e-17,-2.028475e-17,7.121853e-18,1.292403e-17,-6.321830e-18,5047000.0,0.61,0.0
8,2011-12-22,Yum Even better than deep fried butter,165,69,0.250000,-0.075686,0.000404,0.068518,-0.045975,-0.004098,...,-3.601753e-17,5.117519e-17,3.655985e-17,3.055629e-17,8.163704e-18,3.154859e-17,1.574211e-18,5047000.0,0.61,0.0


In [14]:
df_with_tfidf_pca.to_csv('../csv/updated_target_merged_data.csv', index=False)